In [14]:
import tensorflow as tf2
assert tf2.__version__ >= "2.0"
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
from tqdm import tqdm
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Manager
import random
import os
from scipy.integrate import quad_vec  # quad_vec allows to compute integrals accurately
from scipy.stats import norm
from scipy.stats import qmc # to perform Latin Hypercube Sampling (LHS) 
import pandas as pd 

def set_random_seed(seed=42):
    tf.compat.v1.set_random_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress INFO and WARNING logs
tf = tf2.compat.v1
tf.logging.set_verbosity(tf.logging.ERROR)
set_random_seed()
real_type = tf.float32

## Load the dataset
feller_data = pd.read_csv('data//feller_d2.csv')

## Obtain 2nd order differentials and their PCs
second_differentials = feller_data.iloc[:,-36:]
normalized_zscore = (second_differentials - second_differentials.mean()) / second_differentials.std()
pca_d2 = PCA(n_components=15)
second_differential_label = pca_d2.fit_transform(normalized_zscore[:9000])

## Train-test Split (90-10)
feller_testing = feller_data.iloc[9000:]
feller_data = feller_data.iloc[:9000]

training_col = feller_data.columns[:8]
training_target = feller_data.columns[9]
network_inputs = feller_data[training_col].values
network_outputs = feller_data[training_target].values

func_names = ["lm", "r", "tau", "theta", "sigma", "rho", "kappa", "v0"]
network_inputs = feller_data[func_names]
option_prices = feller_data['P_hat']
network_first_order = feller_data[[f"diff wrt {col}" for col in func_names]].values
sec_order_names = []
for i in func_names:
    for j in func_names:
        if os.path.exists(f"data//d2_{i}_{j}.csv"):
            sec_order_names.append(f"d2_{i}_{j}")
network_second_order = feller_data[[f"{col}" for col in sec_order_names]].values

We consider four models:
- Model 1: The benchmark framework 
    
    (Heston parameters $\mapsto$ option price)
- Model 2: Model trained with 1st order differentials 

    (Heston parameters $\mapsto$ (option price & 1st order differentials))
- Model 3: Model trained with 1st and 2nd order differentials 

    (Heston parameters $\mapsto$ (option price & 1st and 2nd differentials))
- Model 4: Model trained with 1st order differentials, and 2nd order Diff-PCA differentials 

    (Heston parameters $\mapsto$ (option price & 1st order differentials, 2nd order Diff-PCA differentials))

Model 1: Without Differentials

In [15]:
def twin_net_with_first_order(hidden_units=64, hidden_layers=3):
    raw_inputs = tf.keras.Input(shape=(8,))
    x = raw_inputs
    # Hidden layers
    for _ in range(hidden_layers):
        x = tf.keras.layers.Dense(hidden_units, activation='softplus')(x)
    # Output layer (option price)
    option_price = tf.keras.layers.Dense(1)(x)  # Predicted option price
    # Create model with two outputs: option price and second-order differential
    model = tf.keras.Model(inputs=raw_inputs, outputs=[option_price])
    return model

# Function to compute gradients and return price, first-order, and second-order differentials
def compute_grad_model1(model, raw_inputs):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(raw_inputs)  # Watch the raw inputs
        option_price = model(raw_inputs)  # Get the outputs (option price, second-order diff)
    
    # Compute the first-order differential (gradient of option price w.r.t raw inputs)
    first_order_differential = tape.gradient(option_price, raw_inputs)
    
    return option_price, first_order_differential

def compute_first_order_differential(model, raw_inputs):
    with tf.GradientTape() as tape:
        tape.watch(raw_inputs)  # Watch the raw inputs
        option_price = model(raw_inputs)[0]  # Get the option price (y_pred[0])
    
    # Compute the first-order differential (gradients w.r.t raw inputs)
    first_order_differential = tape.gradient(option_price, raw_inputs)
    
    return first_order_differential

# Loss function that uses true gradients and predicted gradients
def loss_fn(y_true, y_pred, true_first_order_differentials):
    # Extract the true and predicted option price and second-order differentials
    true_option_price = y_true[0]  # The first element contains the true option price
    pred_option_price = y_pred[0]  # The first element contains the predicted option price
    pred_first_order_diff = y_pred[1]

    # Option price loss (L2 loss)
    price_loss = tf.reduce_mean(tf.square(true_option_price - pred_option_price))  
    # First-order differential loss (L2 loss)
    first_order_loss = tf.reduce_mean(tf.square(true_first_order_differentials - pred_first_order_diff))

    # Total loss (could be a weighted sum of the individual losses)
    total_loss = price_loss + 0.5 * first_order_loss 

    # Return all losses (can be used for monitoring during training)
    return total_loss, price_loss, first_order_loss


In [16]:
# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Generate some random input data (raw inputs)
raw_inputs = tf.convert_to_tensor(network_inputs, dtype=tf.float32)

# Generate some dummy true data for loss calculation (real data would come from a model/simulation)
true_prices = tf.convert_to_tensor(option_prices.values.reshape(1,-1), dtype=tf.float32)
true_first_order_differentials = tf.convert_to_tensor(network_first_order, dtype=tf.float32)
true_second_order_differentials = tf.convert_to_tensor(network_second_order, dtype=tf.float32)

# Create the model
model = twin_net_with_first_order()
history_1 = {
    'total_loss': [],
    'price_loss': [],
    'first_order_loss': [],
    'second_order_loss': []
}

# Training loop
start_time1 = time.time()
for epoch in range(1000):  # Number of epochs
    with tf.GradientTape() as tape:
        # Get predictions (tuple of price, first-order diff, second-order diff)
        predicted_price, predicted_first_order = compute_grad_model1(model, raw_inputs)
        
        # Compute loss (pass the true gradients as part of the loss function)
        total_loss, price_loss, first_order_loss = loss_fn(
            [true_prices, true_second_order_differentials],  # True data
            [predicted_price, predicted_first_order],  # Model predictions
            true_first_order_differentials  # True first-order differentials
        )
    
    # Compute gradients with respect to model parameters
    gradients = tape.gradient(price_loss, model.trainable_variables)
    
    # Update model parameters using the optimizer
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    history_1['total_loss'].append(total_loss.numpy())
    history_1['price_loss'].append(price_loss.numpy())
    history_1['first_order_loss'].append(first_order_loss.numpy())
    
    # Print the loss every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}:")
        print(f"  Total Loss: {total_loss.numpy()}")
        print(f"  Price Loss: {price_loss.numpy()}")
        print(f"  First-Order Loss: {first_order_loss.numpy()}")
end_time1 = time.time()
model.save(f"models//model1.h5")

Epoch 0:
  Total Loss: 2.947831392288208
  Price Loss: 2.8396799564361572
  First-Order Loss: 0.21630285680294037
Epoch 10:
  Total Loss: 0.43661102652549744
  Price Loss: 0.3261074721813202
  First-Order Loss: 0.2210071086883545
Epoch 20:
  Total Loss: 0.30645737051963806
  Price Loss: 0.1964360475540161
  First-Order Loss: 0.2200426608324051
Epoch 30:
  Total Loss: 0.23218996822834015
  Price Loss: 0.12099408358335495
  First-Order Loss: 0.2223917692899704
Epoch 40:
  Total Loss: 0.19681274890899658
  Price Loss: 0.08593636751174927
  First-Order Loss: 0.22175276279449463
Epoch 50:
  Total Loss: 0.18540123105049133
  Price Loss: 0.07414664328098297
  First-Order Loss: 0.22250917553901672
Epoch 60:
  Total Loss: 0.18193572759628296
  Price Loss: 0.07078299671411514
  First-Order Loss: 0.22230544686317444
Epoch 70:
  Total Loss: 0.18104252219200134
  Price Loss: 0.06974344700574875
  First-Order Loss: 0.22259816527366638
Epoch 80:
  Total Loss: 0.18061447143554688
  Price Loss: 0.06933

Model 2: Training with first order differentials

In [17]:
def twin_net_with_first_order(hidden_units=64, hidden_layers=3):
    raw_inputs = tf.keras.Input(shape=(8,))
    x = raw_inputs
    # Hidden layers
    for _ in range(hidden_layers):
        x = tf.keras.layers.Dense(hidden_units, activation='softplus')(x)
    
    # Output layer (option price)
    option_price = tf.keras.layers.Dense(1)(x)  # Predicted option price
    
    # Create model with two outputs: option price and second-order differential
    model = tf.keras.Model(inputs=raw_inputs, outputs=[option_price])
    return model

# Function to compute gradients and return price, first-order, and second-order differentials
def compute_grad_model2(model, raw_inputs):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(raw_inputs)  # Watch the raw inputs
        option_price = model(raw_inputs)  # Get the outputs (option price, second-order diff)
    
    # Compute the first-order differential (gradient of option price w.r.t raw inputs)
    first_order_differential = tape.gradient(option_price, raw_inputs)
    
    return option_price, first_order_differential

def compute_first_order_differential(model, raw_inputs):
    with tf.GradientTape() as tape:
        tape.watch(raw_inputs)  # Watch the raw inputs
        option_price = model(raw_inputs)[0]  # Get the option price (y_pred[0])
    
    # Compute the first-order differential (gradients w.r.t raw inputs)
    first_order_differential = tape.gradient(option_price, raw_inputs)
    
    return first_order_differential
# Loss function that uses true gradients and predicted gradients
def loss_fn(y_true, y_pred, true_first_order_differentials, lambda1):
    # Extract the true and predicted option price and second-order differentials
    true_option_price = y_true[0]  # The first element contains the true option price
    pred_option_price = y_pred[0]  # The first element contains the predicted option price
    pred_first_order_diff = y_pred[1]

    # Option price loss (L2 loss)
    price_loss = tf.reduce_mean(tf.square(true_option_price - pred_option_price))  
    # First-order differential loss (L2 loss)
    first_order_loss = tf.reduce_mean(tf.square(true_first_order_differentials - pred_first_order_diff))
    # Total loss (could be a weighted sum of the individual losses)
    total_loss = price_loss + lambda1 * first_order_loss 
    
    # Return all losses (can be used for monitoring during training)
    return total_loss, price_loss, first_order_loss


In [18]:
# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Generate some random input data (raw inputs)
raw_inputs = tf.convert_to_tensor(network_inputs, dtype=tf.float32)

# Generate some dummy true data for loss calculation (real data would come from a model/simulation)
true_prices = tf.convert_to_tensor(option_prices.values.reshape(1,-1), dtype=tf.float32)
true_first_order_differentials = tf.convert_to_tensor(network_first_order, dtype=tf.float32)
true_second_order_differentials = tf.convert_to_tensor(network_second_order, dtype=tf.float32)

# Create the model
model = twin_net_with_first_order()
history_2 = {
    'total_loss': [],
    'price_loss': [],
    'first_order_loss': [],
    'second_order_loss': [],
    'lambda1': [],
}
start_time2 = time.time()
# Training loop
for lambda1 in [0.1,0.5,0.7]:
    for epoch in range(1000):  # Number of epochs
        with tf.GradientTape() as tape:
            # Get predictions (tuple of price, first-order diff, second-order diff)
            predicted_price, predicted_first_order = compute_grad_model2(model, raw_inputs)
            
            # Compute loss (pass the true gradients as part of the loss function)
            total_loss, price_loss, first_order_loss = loss_fn(
                [true_prices, true_second_order_differentials],  # True data
                [predicted_price, predicted_first_order],  # Model predictions
                true_first_order_differentials,  # True first-order differentials
                lambda1
            )
        
        # Compute gradients with respect to model parameters
        gradients = tape.gradient(total_loss, model.trainable_variables)
        
        # Update model parameters using the optimizer
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        history_2['total_loss'].append(total_loss.numpy())
        history_2['price_loss'].append(price_loss.numpy())
        history_2['first_order_loss'].append(first_order_loss.numpy())
        history_2['lambda1'].append(lambda1)
        # Print the loss every 10 epochs
        if epoch % 100 == 0:
            print(f"Epoch {epoch}:")
            print(f"  Total Loss: {total_loss.numpy()}")
            print(f"  Price Loss: {price_loss.numpy()}")
            print(f"  First-Order Loss: {first_order_loss.numpy()}")
    model.save(f"models//model2_{int(lambda1*10)}.h5")
end_time2 = time.time()

Epoch 0:
  Total Loss: 4.972015380859375
  Price Loss: 4.9492669105529785
  First-Order Loss: 0.2274853140115738
Epoch 100:
  Total Loss: 0.09233073890209198
  Price Loss: 0.06949058920145035
  First-Order Loss: 0.22840149700641632
Epoch 200:
  Total Loss: 0.09059886634349823
  Price Loss: 0.0691138505935669
  First-Order Loss: 0.21485011279582977
Epoch 300:
  Total Loss: 0.0887564867734909
  Price Loss: 0.06890406459569931
  First-Order Loss: 0.19852417707443237
Epoch 400:
  Total Loss: 0.08665446192026138
  Price Loss: 0.06878771632909775
  First-Order Loss: 0.17866745591163635
Epoch 500:
  Total Loss: 0.08415982127189636
  Price Loss: 0.06875917315483093
  First-Order Loss: 0.15400651097297668
Epoch 600:
  Total Loss: 0.08053094148635864
  Price Loss: 0.06901604682207108
  First-Order Loss: 0.11514895409345627
Epoch 700:
  Total Loss: 0.07840901613235474
  Price Loss: 0.06972654908895493
  First-Order Loss: 0.08682466298341751
Epoch 800:
  Total Loss: 0.07786636054515839
  Price Los

Epoch 0:
  Total Loss: 0.10764767229557037
  Price Loss: 0.07002219557762146
  First-Order Loss: 0.07525096088647842
Epoch 100:
  Total Loss: 0.10219007730484009
  Price Loss: 0.07296060770750046
  First-Order Loss: 0.05845893174409866
Epoch 200:
  Total Loss: 0.10043349862098694
  Price Loss: 0.0734710618853569
  First-Order Loss: 0.05392488092184067
Epoch 300:
  Total Loss: 0.09868831932544708
  Price Loss: 0.07374423742294312
  First-Order Loss: 0.04988816753029823
Epoch 400:
  Total Loss: 0.09695909172296524
  Price Loss: 0.07405951619148254
  First-Order Loss: 0.04579915478825569
Epoch 500:
  Total Loss: 0.09528401494026184
  Price Loss: 0.07432039082050323
  First-Order Loss: 0.04192724451422691
Epoch 600:
  Total Loss: 0.09379999339580536
  Price Loss: 0.07449774444103241
  First-Order Loss: 0.0386045016348362
Epoch 700:
  Total Loss: 0.09262506663799286
  Price Loss: 0.07457904517650604
  First-Order Loss: 0.03609205037355423
Epoch 800:
  Total Loss: 0.09163597226142883
  Price

Epoch 0:
  Total Loss: 0.09583663195371628
  Price Loss: 0.07461155205965042
  First-Order Loss: 0.030321545898914337
Epoch 100:
  Total Loss: 0.09411412477493286
  Price Loss: 0.07604441046714783
  First-Order Loss: 0.02581387758255005
Epoch 200:
  Total Loss: 0.09290707111358643
  Price Loss: 0.07605723291635513
  First-Order Loss: 0.024071194231510162
Epoch 300:
  Total Loss: 0.09187775105237961
  Price Loss: 0.0760183185338974
  First-Order Loss: 0.022656330838799477
Epoch 400:
  Total Loss: 0.09095241874456406
  Price Loss: 0.07597412168979645
  First-Order Loss: 0.02139757014811039
Epoch 500:
  Total Loss: 0.09009183198213577
  Price Loss: 0.07593785971403122
  First-Order Loss: 0.020219963043928146
Epoch 600:
  Total Loss: 0.08930762112140656
  Price Loss: 0.07591329514980316
  First-Order Loss: 0.019134752452373505
Epoch 700:
  Total Loss: 0.08868751674890518
  Price Loss: 0.07584922015666962
  First-Order Loss: 0.018340427428483963
Epoch 800:
  Total Loss: 0.08817845582962036


Model 3: Training with second order differentials without Diff-PCA

In [19]:

def twin_net_with_first_second_order(input_dim, hidden_units=64, hidden_layers=3):
    raw_inputs = tf.keras.Input(shape=(8,))
    x = raw_inputs

    # Hidden layers
    for _ in range(hidden_layers):
        x = tf.keras.layers.Dense(hidden_units, activation='softplus')(x)
    
    # Output layer (option price)
    option_price = tf.keras.layers.Dense(1)(x)  # Predicted option price
    second_order_diff = tf.keras.layers.Dense(input_dim)(x)  # Predicted second-order differential (36 elements)
    
    # Create model with two outputs: option price and second-order differential
    model = tf.keras.Model(inputs=raw_inputs, outputs=[option_price, second_order_diff])
    return model

# Function to compute gradients and return price, first-order, and second-order differentials
def compute_grad_model3(model, raw_inputs):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(raw_inputs)  # Watch the raw inputs
        option_price, second_order_diff = model(raw_inputs)  # Get the outputs (option price, second-order diff)
    
    # Compute the first-order differential (gradient of option price w.r.t raw inputs)
    first_order_differential = tape.gradient(option_price, raw_inputs)
    
    return option_price, first_order_differential, second_order_diff

def compute_first_order_differential(model, raw_inputs):
    with tf.GradientTape() as tape:
        tape.watch(raw_inputs)  # Watch the raw inputs
        option_price = model(raw_inputs)[0]  # Get the option price (y_pred[0])
    
    # Compute the first-order differential (gradients w.r.t raw inputs)
    first_order_differential = tape.gradient(option_price, raw_inputs)
    
    return first_order_differential
# Loss function that uses true gradients and predicted gradients
def loss_fn(y_true, y_pred, true_first_order_differentials, lambda1, lambda2):
    # Extract the true and predicted option price and second-order differentials
    true_option_price = y_true[0]  # The first element contains the true option price
    true_second_order_diff = y_true[1]  # The remaining elements contain the true second-order differentials
    pred_option_price = y_pred[0]  # The first element contains the predicted option price
    pred_second_order_diff = y_pred[2]  # The second element contains the predicted second-order differentials
    pred_first_order_diff = y_pred[1]
    # Option price loss (L2 loss)
    price_loss = tf.reduce_mean(tf.square(true_option_price - pred_option_price))  
    # Second-order differential loss (L2 loss)
    second_order_loss = tf.reduce_mean(tf.square(true_second_order_diff - pred_second_order_diff))
    
    # First-order differential loss (L2 loss)
    first_order_loss = tf.reduce_mean(tf.square(true_first_order_differentials - pred_first_order_diff))

    # Total loss (could be a weighted sum of the individual losses)
    total_loss = price_loss + lambda1 * first_order_loss + lambda2* second_order_loss
    
    # Return all losses (can be used for monitoring during training)
    return total_loss, price_loss, first_order_loss, second_order_loss


In [20]:
# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Generate some random input data (raw inputs)
raw_inputs = tf.convert_to_tensor(network_inputs, dtype=tf.float32)

# Generate some dummy true data for loss calculation (real data would come from a model/simulation)
true_prices = tf.convert_to_tensor(option_prices.values.reshape(1,-1), dtype=tf.float32)
true_first_order_differentials = tf.convert_to_tensor(network_first_order, dtype=tf.float32)
true_second_order_differentials = tf.convert_to_tensor(network_second_order, dtype=tf.float32)

# Create the model
model = twin_net_with_first_second_order(network_second_order.shape[1])
history_3 = {
    'total_loss': [],
    'price_loss': [],
    'first_order_loss': [],
    'second_order_loss': [],
    'lambda1': [],
    'lambda2': []
}
start_time3 = time.time()
# Training loop
for lambda1 in [0.1,0.5,0.7]:
    for lambda2 in [0.1,0.5,0.7]:
        for epoch in range(1000):  # Number of epochs
            with tf.GradientTape() as tape:
                # Get predictions (tuple of price, first-order diff, second-order diff)
                predicted_price, predicted_first_order, predicted_second_order = compute_grad_model3(model, raw_inputs)
                
                # Compute loss (pass the true gradients as part of the loss function)
                total_loss, price_loss, first_order_loss, second_order_loss = loss_fn(
                    [true_prices, true_second_order_differentials],  # True data
                    [predicted_price, predicted_first_order, predicted_second_order],  # Model predictions
                    true_first_order_differentials,  # True first-order differentials
                    lambda1, lambda2
                )
            
            # Compute gradients with respect to model parameters
            gradients = tape.gradient(total_loss, model.trainable_variables)
            
            # Update model parameters using the optimizer
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            history_3['total_loss'].append(total_loss.numpy())
            history_3['price_loss'].append(price_loss.numpy())
            history_3['first_order_loss'].append(first_order_loss.numpy())
            history_3['second_order_loss'].append(second_order_loss.numpy())
            history_3['lambda1'].append(lambda1)
            history_3['lambda2'].append(lambda2)
            # Print the loss every 10 epochs
            if epoch % 100 == 0:
                print(f"Epoch {epoch}:")
                print(f"  Total Loss: {total_loss.numpy()}")
                print(f"  Price Loss: {price_loss.numpy()}")
                print(f"  First-Order Loss: {first_order_loss.numpy()}")
                print(f"  Second-Order Loss: {second_order_loss.numpy()}")
        model.save(f"models//model3_{int(lambda1*10)}_{int(lambda2*10)}.h5")

end_time3 = time.time()

Epoch 0:
  Total Loss: 16.60838508605957
  Price Loss: 16.307416915893555
  First-Order Loss: 0.23686927556991577
  Second-Order Loss: 2.772812604904175
Epoch 100:
  Total Loss: 0.18743860721588135
  Price Loss: 0.07054278254508972
  First-Order Loss: 0.2305145412683487
  Second-Order Loss: 0.938443660736084
Epoch 200:
  Total Loss: 0.18272101879119873
  Price Loss: 0.06949366629123688
  First-Order Loss: 0.21727417409420013
  Second-Order Loss: 0.9149993062019348
Epoch 300:
  Total Loss: 0.17841263115406036
  Price Loss: 0.06910976767539978
  First-Order Loss: 0.20208723843097687
  Second-Order Loss: 0.8909413814544678
Epoch 400:
  Total Loss: 0.17408785223960876
  Price Loss: 0.06898214668035507
  First-Order Loss: 0.18570838868618011
  Second-Order Loss: 0.8653485774993896
Epoch 500:
  Total Loss: 0.16978618502616882
  Price Loss: 0.06897207349538803
  First-Order Loss: 0.16848352551460266
  Second-Order Loss: 0.8396574854850769
Epoch 600:
  Total Loss: 0.1658017635345459
  Price Lo

Epoch 0:
  Total Loss: 0.46562570333480835
  Price Loss: 0.06949429959058762
  First-Order Loss: 0.09557302296161652
  Second-Order Loss: 0.7731481790542603
Epoch 100:
  Total Loss: 0.45477283000946045
  Price Loss: 0.06985919922590256
  First-Order Loss: 0.08635707199573517
  Second-Order Loss: 0.7525558471679688
Epoch 200:
  Total Loss: 0.4227628707885742
  Price Loss: 0.0708736777305603
  First-Order Loss: 0.07599873840808868
  Second-Order Loss: 0.6885786056518555
Epoch 300:
  Total Loss: 0.36473968625068665
  Price Loss: 0.07110771536827087
  First-Order Loss: 0.06977252662181854
  Second-Order Loss: 0.5733094215393066
Epoch 400:
  Total Loss: 0.29333269596099854
  Price Loss: 0.07119731605052948
  First-Order Loss: 0.05832584574818611
  Second-Order Loss: 0.4326056241989136
Epoch 500:
  Total Loss: 0.25332602858543396
  Price Loss: 0.07123824954032898
  First-Order Loss: 0.05089506134390831
  Second-Order Loss: 0.35399654507637024
Epoch 600:
  Total Loss: 0.2323598563671112
  Pri

Epoch 0:
  Total Loss: 0.24120543897151947
  Price Loss: 0.07081835716962814
  First-Order Loss: 0.045344278216362
  Second-Order Loss: 0.23693236708641052
Epoch 100:
  Total Loss: 0.23458105325698853
  Price Loss: 0.07075230777263641
  First-Order Loss: 0.04544878378510475
  Second-Order Loss: 0.22754839062690735
Epoch 200:
  Total Loss: 0.22904103994369507
  Price Loss: 0.07074358314275742
  First-Order Loss: 0.04468289017677307
  Second-Order Loss: 0.21975596249103546
Epoch 300:
  Total Loss: 0.2238970398902893
  Price Loss: 0.07073578238487244
  First-Order Loss: 0.043559130281209946
  Second-Order Loss: 0.21257907152175903
Epoch 400:
  Total Loss: 0.21892423927783966
  Price Loss: 0.07071542739868164
  First-Order Loss: 0.042064912617206573
  Second-Order Loss: 0.20571759343147278
Epoch 500:
  Total Loss: 0.21384501457214355
  Price Loss: 0.07067383080720901
  First-Order Loss: 0.04018335044384003
  Second-Order Loss: 0.1987897902727127
Epoch 600:
  Total Loss: 0.20823830366134644

Epoch 0:
  Total Loss: 0.10533587634563446
  Price Loss: 0.07194730639457703
  First-Order Loss: 0.03493237495422363
  Second-Order Loss: 0.15922382473945618
Epoch 100:
  Total Loss: 0.09858434647321701
  Price Loss: 0.07396739721298218
  First-Order Loss: 0.01785958930850029
  Second-Order Loss: 0.15687152743339539
Epoch 200:
  Total Loss: 0.09808387607336044
  Price Loss: 0.07398980855941772
  First-Order Loss: 0.017059870064258575
  Second-Order Loss: 0.1556413322687149
Epoch 300:
  Total Loss: 0.09772505611181259
  Price Loss: 0.07402398437261581
  First-Order Loss: 0.016548005864024162
  Second-Order Loss: 0.15427066385746002
Epoch 400:
  Total Loss: 0.09736863523721695
  Price Loss: 0.07400354743003845
  First-Order Loss: 0.01617722399532795
  Second-Order Loss: 0.15276473760604858
Epoch 500:
  Total Loss: 0.09708643704652786
  Price Loss: 0.07403602451086044
  First-Order Loss: 0.01586928963661194
  Second-Order Loss: 0.1511576771736145
Epoch 600:
  Total Loss: 0.096751913428306

Epoch 0:
  Total Loss: 0.15236660838127136
  Price Loss: 0.07395646721124649
  First-Order Loss: 0.014793340116739273
  Second-Order Loss: 0.14202696084976196
Epoch 100:
  Total Loss: 0.1489308774471283
  Price Loss: 0.0739184319972992
  First-Order Loss: 0.014767259359359741
  Second-Order Loss: 0.13525761663913727
Epoch 200:
  Total Loss: 0.14529795944690704
  Price Loss: 0.07391978055238724
  First-Order Loss: 0.014591642655432224
  Second-Order Loss: 0.12816470861434937
Epoch 300:
  Total Loss: 0.14408184587955475
  Price Loss: 0.07608912140130997
  First-Order Loss: 0.014499376527965069
  Second-Order Loss: 0.12148606032133102
Epoch 400:
  Total Loss: 0.13831354677677155
  Price Loss: 0.0738995298743248
  First-Order Loss: 0.014207322150468826
  Second-Order Loss: 0.11462070792913437
Epoch 500:
  Total Loss: 0.13496901094913483
  Price Loss: 0.07388337701559067
  First-Order Loss: 0.013994138687849045
  Second-Order Loss: 0.10817713290452957
Epoch 600:
  Total Loss: 0.132398679852

Epoch 0:
  Total Loss: 0.1371280550956726
  Price Loss: 0.07378915697336197
  First-Order Loss: 0.01324229221791029
  Second-Order Loss: 0.0810253694653511
Epoch 100:
  Total Loss: 0.1353522539138794
  Price Loss: 0.07380296289920807
  First-Order Loss: 0.013182012364268303
  Second-Order Loss: 0.07851184159517288
Epoch 200:
  Total Loss: 0.13340413570404053
  Price Loss: 0.0737837627530098
  First-Order Loss: 0.013154134154319763
  Second-Order Loss: 0.07577615231275558
Epoch 300:
  Total Loss: 0.13158677518367767
  Price Loss: 0.0737793818116188
  First-Order Loss: 0.013117275200784206
  Second-Order Loss: 0.07321251183748245
Epoch 400:
  Total Loss: 0.1298571527004242
  Price Loss: 0.07377533614635468
  First-Order Loss: 0.013086647726595402
  Second-Order Loss: 0.07076926529407501
Epoch 500:
  Total Loss: 0.13474400341510773
  Price Loss: 0.08038923889398575
  First-Order Loss: 0.012503607198596
  Second-Order Loss: 0.06871850788593292
Epoch 600:
  Total Loss: 0.12668925523757935
 

Epoch 0:
  Total Loss: 0.08861841261386871
  Price Loss: 0.07374989986419678
  First-Order Loss: 0.012878085486590862
  Second-Order Loss: 0.05853855609893799
Epoch 100:
  Total Loss: 0.08818250149488449
  Price Loss: 0.07494806498289108
  First-Order Loss: 0.010552686639130116
  Second-Order Loss: 0.05847559869289398
Epoch 200:
  Total Loss: 0.08812522888183594
  Price Loss: 0.07497003674507141
  First-Order Loss: 0.010450538247823715
  Second-Order Loss: 0.05839816480875015
Epoch 300:
  Total Loss: 0.08796286582946777
  Price Loss: 0.07495608925819397
  First-Order Loss: 0.010267432779073715
  Second-Order Loss: 0.05819573998451233
Epoch 400:
  Total Loss: 0.08787230402231216
  Price Loss: 0.07495414465665817
  First-Order Loss: 0.010170974768698215
  Second-Order Loss: 0.05798474699258804
Epoch 500:
  Total Loss: 0.08780618757009506
  Price Loss: 0.07494296133518219
  First-Order Loss: 0.010116441175341606
  Second-Order Loss: 0.05781714618206024
Epoch 600:
  Total Loss: 0.087718136

Epoch 0:
  Total Loss: 0.10997393727302551
  Price Loss: 0.07493752986192703
  First-Order Loss: 0.00978799071162939
  Second-Order Loss: 0.05636963993310928
Epoch 100:
  Total Loss: 0.1091671958565712
  Price Loss: 0.07495688647031784
  First-Order Loss: 0.009965462610125542
  Second-Order Loss: 0.05446897819638252
Epoch 200:
  Total Loss: 0.10813149809837341
  Price Loss: 0.07493934780359268
  First-Order Loss: 0.009839970618486404
  Second-Order Loss: 0.05260834842920303
Epoch 300:
  Total Loss: 0.10728591680526733
  Price Loss: 0.07493963837623596
  First-Order Loss: 0.00981123000383377
  Second-Order Loss: 0.050956834107637405
Epoch 400:
  Total Loss: 0.10655447840690613
  Price Loss: 0.07494175434112549
  First-Order Loss: 0.009801547974348068
  Second-Order Loss: 0.04950328543782234
Epoch 500:
  Total Loss: 0.10575554519891739
  Price Loss: 0.07493893057107925
  First-Order Loss: 0.009740649722516537
  Second-Order Loss: 0.04799632728099823
Epoch 600:
  Total Loss: 0.10497523844

Epoch 0:
  Total Loss: 0.11023001372814178
  Price Loss: 0.07496275752782822
  First-Order Loss: 0.009492255747318268
  Second-Order Loss: 0.04088953882455826
Epoch 100:
  Total Loss: 0.10961588472127914
  Price Loss: 0.0749463364481926
  First-Order Loss: 0.009497448801994324
  Second-Order Loss: 0.04003048315644264
Epoch 200:
  Total Loss: 0.10882974416017532
  Price Loss: 0.07495821267366409
  First-Order Loss: 0.009454194456338882
  Second-Order Loss: 0.03893371298909187
Epoch 300:
  Total Loss: 0.10806334763765335
  Price Loss: 0.07496213167905807
  First-Order Loss: 0.009416607208549976
  Second-Order Loss: 0.03787084296345711
Epoch 400:
  Total Loss: 0.10734463483095169
  Price Loss: 0.0749896764755249
  First-Order Loss: 0.009352732449769974
  Second-Order Loss: 0.036868635565042496
Epoch 500:
  Total Loss: 0.10654999315738678
  Price Loss: 0.07496847212314606
  First-Order Loss: 0.009339374490082264
  Second-Order Loss: 0.035777077078819275
Epoch 600:
  Total Loss: 0.105747990

Model 4: Training with second order differentials with Diff-PCA

In [21]:

def twin_net_with_first_second_order(input_dim, hidden_units=64, hidden_layers=3):
    raw_inputs = tf.keras.Input(shape=(8,))
    x = raw_inputs

    # Hidden layers
    for _ in range(hidden_layers):
        x = tf.keras.layers.Dense(hidden_units, activation='softplus')(x)
    
    # Output layer (option price)
    option_price = tf.keras.layers.Dense(1)(x)  # Predicted option price
    second_order_diff = tf.keras.layers.Dense(input_dim)(x)  # Predicted second-order differential (36 elements)
    
    # Create model with two outputs: option price and second-order differential
    model = tf.keras.Model(inputs=raw_inputs, outputs=[option_price, second_order_diff])
    return model

# Function to compute gradients and return price, first-order, and second-order differentials
def compute_grad_model4(model, raw_inputs):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(raw_inputs)  # Watch the raw inputs
        option_price, second_order_diff = model(raw_inputs)  # Get the outputs (option price, second-order diff)
    
    # Compute the first-order differential (gradient of option price w.r.t raw inputs)
    first_order_differential = tape.gradient(option_price, raw_inputs)
    
    return option_price, first_order_differential, second_order_diff

def compute_first_order_differential(model, raw_inputs):
    with tf.GradientTape() as tape:
        tape.watch(raw_inputs)  # Watch the raw inputs
        option_price = model(raw_inputs)[0]  # Get the option price (y_pred[0])
    
    # Compute the first-order differential (gradients w.r.t raw inputs)
    first_order_differential = tape.gradient(option_price, raw_inputs)
    
    return first_order_differential
# Loss function that uses true gradients and predicted gradients
def loss_fn(y_true, y_pred, true_first_order_differentials, lambda1, lambda2):
    # Extract the true and predicted option price and second-order differentials
    true_option_price = y_true[0]  # The first element contains the true option price
    true_second_order_diff = y_true[1]  # The remaining elements contain the true second-order differentials
    pred_option_price = y_pred[0]  # The first element contains the predicted option price
    pred_second_order_diff = y_pred[2]  # The second element contains the predicted second-order differentials
    pred_first_order_diff = y_pred[1]
    # Option price loss (L2 loss)
    price_loss = tf.reduce_mean(tf.square(true_option_price - pred_option_price))  
    
    # print(true_second_order_diff,pred_second_order_diff)
    # Second-order differential loss (L2 loss)
    second_order_loss = tf.reduce_mean(tf.square(true_second_order_diff - pred_second_order_diff))
    
    # First-order differential loss (L2 loss)
    first_order_loss = tf.reduce_mean(tf.square(true_first_order_differentials - pred_first_order_diff))
    # Total loss (could be a weighted sum of the individual losses)
    total_loss = price_loss + lambda1 * first_order_loss + lambda2*second_order_loss
    
    # Return all losses (can be used for monitoring during training)
    return total_loss, price_loss, first_order_loss, second_order_loss


In [22]:
# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Generate some random input data (raw inputs)
raw_inputs = tf.convert_to_tensor(network_inputs, dtype=tf.float32)

# Generate some dummy true data for loss calculation (real data would come from a model/simulation)
true_prices = tf.convert_to_tensor(option_prices.values.reshape(1,-1), dtype=tf.float32)
true_first_order_differentials = tf.convert_to_tensor(network_first_order, dtype=tf.float32)
true_second_order_differentials = tf.convert_to_tensor(second_differential_label, dtype=tf.float32)

# Create the model
model = twin_net_with_first_second_order(second_differential_label.shape[1])
history_4 = {
    'total_loss': [],
    'price_loss': [],
    'first_order_loss': [],
    'second_order_loss': [],
    'lambda1': [],
    'lambda2': []
}

start_time4 = time.time()
# Training loop
for lambda1 in [0.1,0.5,0.7]:
    for lambda2 in [0.1,0.5,0.7]:
        for epoch in range(1000):  # Number of epochs
            with tf.GradientTape() as tape:
                # Get predictions (tuple of price, first-order diff, second-order diff)
                predicted_price, predicted_first_order, predicted_second_order = compute_grad_model4(model, raw_inputs)
                
                # Compute loss (pass the true gradients as part of the loss function)
                total_loss, price_loss, first_order_loss, second_order_loss = loss_fn(
                    [true_prices, true_second_order_differentials],  # True data
                    [predicted_price, predicted_first_order, predicted_second_order],  # Model predictions
                    true_first_order_differentials,  # True first-order differentials
                    lambda1, lambda2
                )
            
            # Compute gradients with respect to model parameters
            gradients = tape.gradient(total_loss, model.trainable_variables)
            
            # Update model parameters using the optimizer
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            history_4['total_loss'].append(total_loss.numpy())
            history_4['price_loss'].append(price_loss.numpy())
            history_4['first_order_loss'].append(first_order_loss.numpy())
            history_4['second_order_loss'].append(second_order_loss.numpy())
            history_4['lambda1'].append(lambda1)
            history_4['lambda2'].append(lambda2)
            # Print the loss every 10 epochs
            if epoch % 100 == 0:
                print(f"Epoch {epoch}:")
                print(f"  Total Loss: {total_loss.numpy()}")
                print(f"  Price Loss: {price_loss.numpy()}")
                print(f"  First-Order Loss: {first_order_loss.numpy()}")
                print(f"  Second-Order Loss: {second_order_loss.numpy()}")
        model.save(f"models//model4_{int(lambda1*10)}_{int(lambda2*10)}.h5")

end_time4 = time.time()

Epoch 0:
  Total Loss: 5.415454387664795
  Price Loss: 5.060916423797607
  First-Order Loss: 0.2286314070224762
  Second-Order Loss: 3.3167495727539062
Epoch 100:
  Total Loss: 0.3038875460624695
  Price Loss: 0.07005780190229416
  First-Order Loss: 0.21640489995479584
  Second-Order Loss: 2.1218926906585693
Epoch 200:
  Total Loss: 0.2944170832633972
  Price Loss: 0.06895658373832703
  First-Order Loss: 0.19681070744991302
  Second-Order Loss: 2.0577940940856934
Epoch 300:
  Total Loss: 0.28534597158432007
  Price Loss: 0.06889110803604126
  First-Order Loss: 0.17232616245746613
  Second-Order Loss: 1.9922224283218384
Epoch 400:
  Total Loss: 0.2756156921386719
  Price Loss: 0.06904451549053192
  First-Order Loss: 0.1429424285888672
  Second-Order Loss: 1.9227691888809204
Epoch 500:
  Total Loss: 0.26738816499710083
  Price Loss: 0.06923498213291168
  First-Order Loss: 0.11212951689958572
  Second-Order Loss: 1.8694024085998535
Epoch 600:
  Total Loss: 0.2592482566833496
  Price Loss:

Epoch 0:
  Total Loss: 0.8095589876174927
  Price Loss: 0.07067367434501648
  First-Order Loss: 0.06921975314617157
  Second-Order Loss: 1.4639266729354858
Epoch 100:
  Total Loss: 0.6820535063743591
  Price Loss: 0.07104717940092087
  First-Order Loss: 0.06602457165718079
  Second-Order Loss: 1.2088077068328857
Epoch 200:
  Total Loss: 0.5841275453567505
  Price Loss: 0.07145591080188751
  First-Order Loss: 0.06399435549974442
  Second-Order Loss: 1.0125443935394287
Epoch 300:
  Total Loss: 0.509914219379425
  Price Loss: 0.07128431648015976
  First-Order Loss: 0.05755561590194702
  Second-Order Loss: 0.8657487034797668
Epoch 400:
  Total Loss: 0.4581071436405182
  Price Loss: 0.07148351520299911
  First-Order Loss: 0.050652485340833664
  Second-Order Loss: 0.7631167769432068
Epoch 500:
  Total Loss: 0.4173237085342407
  Price Loss: 0.07140842080116272
  First-Order Loss: 0.04746885225176811
  Second-Order Loss: 0.6823368072509766
Epoch 600:
  Total Loss: 0.38380569219589233
  Price L

Epoch 0:
  Total Loss: 0.39298343658447266
  Price Loss: 0.07217571884393692
  First-Order Loss: 0.041536204516887665
  Second-Order Loss: 0.452362984418869
Epoch 100:
  Total Loss: 0.37589386105537415
  Price Loss: 0.07127175480127335
  First-Order Loss: 0.04092201218008995
  Second-Order Loss: 0.42932841181755066
Epoch 200:
  Total Loss: 0.3618488311767578
  Price Loss: 0.07131186872720718
  First-Order Loss: 0.040575940161943436
  Second-Order Loss: 0.409256249666214
Epoch 300:
  Total Loss: 0.3488815724849701
  Price Loss: 0.07134002447128296
  First-Order Loss: 0.040218573063611984
  Second-Order Loss: 0.3907424211502075
Epoch 400:
  Total Loss: 0.3371224105358124
  Price Loss: 0.0714135617017746
  First-Order Loss: 0.039740100502967834
  Second-Order Loss: 0.3739069104194641
Epoch 500:
  Total Loss: 0.32602688670158386
  Price Loss: 0.07133495062589645
  First-Order Loss: 0.039613544940948486
  Second-Order Loss: 0.3581865429878235
Epoch 600:
  Total Loss: 0.3155188262462616
  Pr

Epoch 0:
  Total Loss: 0.12256070226430893
  Price Loss: 0.07406952977180481
  First-Order Loss: 0.038666605949401855
  Second-Order Loss: 0.2915787100791931
Epoch 100:
  Total Loss: 0.11260583996772766
  Price Loss: 0.07458159327507019
  First-Order Loss: 0.018110862001776695
  Second-Order Loss: 0.2896881401538849
Epoch 200:
  Total Loss: 0.1116788387298584
  Price Loss: 0.07454212009906769
  First-Order Loss: 0.016549302265048027
  Second-Order Loss: 0.28862065076828003
Epoch 300:
  Total Loss: 0.11102194339036942
  Price Loss: 0.07448483258485794
  First-Order Loss: 0.01565905660390854
  Second-Order Loss: 0.28707587718963623
Epoch 400:
  Total Loss: 0.11047685146331787
  Price Loss: 0.07443301379680634
  First-Order Loss: 0.015034401789307594
  Second-Order Loss: 0.28526636958122253
Epoch 500:
  Total Loss: 0.10999131947755814
  Price Loss: 0.07438734918832779
  First-Order Loss: 0.014554533176124096
  Second-Order Loss: 0.2832670211791992
Epoch 600:
  Total Loss: 0.11199507117271

Epoch 0:
  Total Loss: 0.21657982468605042
  Price Loss: 0.07421743869781494
  First-Order Loss: 0.01314379833638668
  Second-Order Loss: 0.2715809643268585
Epoch 100:
  Total Loss: 0.21059344708919525
  Price Loss: 0.0741785317659378
  First-Order Loss: 0.013623572885990143
  Second-Order Loss: 0.25920626521110535
Epoch 200:
  Total Loss: 0.2049277126789093
  Price Loss: 0.07418887317180634
  First-Order Loss: 0.013601100072264671
  Second-Order Loss: 0.24787656962871552
Epoch 300:
  Total Loss: 0.20111262798309326
  Price Loss: 0.07527177780866623
  First-Order Loss: 0.01346707995980978
  Second-Order Loss: 0.2382146120071411
Epoch 400:
  Total Loss: 0.19465933740139008
  Price Loss: 0.07415597885847092
  First-Order Loss: 0.013413947075605392
  Second-Order Loss: 0.22759276628494263
Epoch 500:
  Total Loss: 0.18987435102462769
  Price Loss: 0.07413391768932343
  First-Order Loss: 0.013300172984600067
  Second-Order Loss: 0.21818067133426666
Epoch 600:
  Total Loss: 0.185401588678359

Epoch 0:
  Total Loss: 0.20572888851165771
  Price Loss: 0.07405777275562286
  First-Order Loss: 0.012744304724037647
  Second-Order Loss: 0.17899851500988007
Epoch 100:
  Total Loss: 0.20231395959854126
  Price Loss: 0.07402405887842178
  First-Order Loss: 0.012805060483515263
  Second-Order Loss: 0.1741248220205307
Epoch 200:
  Total Loss: 0.1988867074251175
  Price Loss: 0.0739976018667221
  First-Order Loss: 0.012832527980208397
  Second-Order Loss: 0.16924692690372467
Epoch 300:
  Total Loss: 0.19557788968086243
  Price Loss: 0.07399479299783707
  First-Order Loss: 0.012801923789083958
  Second-Order Loss: 0.16454589366912842
Epoch 400:
  Total Loss: 0.19232068955898285
  Price Loss: 0.07399113476276398
  First-Order Loss: 0.012762532569468021
  Second-Order Loss: 0.1599261313676834
Epoch 500:
  Total Loss: 0.18913215398788452
  Price Loss: 0.07397907227277756
  First-Order Loss: 0.012778379954397678
  Second-Order Loss: 0.15537700057029724
Epoch 600:
  Total Loss: 0.1860048770904

Epoch 0:
  Total Loss: 0.09611503779888153
  Price Loss: 0.07395176589488983
  First-Order Loss: 0.012479756027460098
  Second-Order Loss: 0.1342744678258896
Epoch 100:
  Total Loss: 0.09548930078744888
  Price Loss: 0.07518419623374939
  First-Order Loss: 0.009849155321717262
  Second-Order Loss: 0.13410697877407074
Epoch 200:
  Total Loss: 0.09527701139450073
  Price Loss: 0.07517363131046295
  First-Order Loss: 0.009602120146155357
  Second-Order Loss: 0.13381895422935486
Epoch 300:
  Total Loss: 0.09511610865592957
  Price Loss: 0.0751543939113617
  First-Order Loss: 0.009448033757507801
  Second-Order Loss: 0.13348090648651123
Epoch 400:
  Total Loss: 0.09496822953224182
  Price Loss: 0.07514677196741104
  First-Order Loss: 0.00932348519563675
  Second-Order Loss: 0.13295017182826996
Epoch 500:
  Total Loss: 0.0948345735669136
  Price Loss: 0.07513400167226791
  First-Order Loss: 0.009229372255504131
  Second-Order Loss: 0.13240008056163788
Epoch 600:
  Total Loss: 0.0948338136076

Epoch 0:
  Total Loss: 0.1459009051322937
  Price Loss: 0.07508710771799088
  First-Order Loss: 0.00892452523112297
  Second-Order Loss: 0.12913325428962708
Epoch 100:
  Total Loss: 0.1438055783510208
  Price Loss: 0.07507383823394775
  First-Order Loss: 0.009110788814723492
  Second-Order Loss: 0.12470836192369461
Epoch 200:
  Total Loss: 0.14189717173576355
  Price Loss: 0.07508136332035065
  First-Order Loss: 0.009137630462646484
  Second-Order Loss: 0.1208389401435852
Epoch 300:
  Total Loss: 0.1414189636707306
  Price Loss: 0.07593665271997452
  First-Order Loss: 0.009538665413856506
  Second-Order Loss: 0.11761046946048737
Epoch 400:
  Total Loss: 0.14049950242042542
  Price Loss: 0.07584960758686066
  First-Order Loss: 0.008467702195048332
  Second-Order Loss: 0.11744502186775208
Epoch 500:
  Total Loss: 0.13709145784378052
  Price Loss: 0.07524172961711884
  First-Order Loss: 0.009246045723557472
  Second-Order Loss: 0.11075498163700104
Epoch 600:
  Total Loss: 0.13527339696884

Epoch 0:
  Total Loss: 0.14939069747924805
  Price Loss: 0.07511866837739944
  First-Order Loss: 0.00896090641617775
  Second-Order Loss: 0.09714198857545853
Epoch 100:
  Total Loss: 0.1482207477092743
  Price Loss: 0.07509689033031464
  First-Order Loss: 0.009039786644279957
  Second-Order Loss: 0.09542287141084671
Epoch 200:
  Total Loss: 0.14695605635643005
  Price Loss: 0.07508392632007599
  First-Order Loss: 0.009065416641533375
  Second-Order Loss: 0.09360906481742859
Epoch 300:
  Total Loss: 0.14574286341667175
  Price Loss: 0.07508233934640884
  First-Order Loss: 0.00906903762370348
  Second-Order Loss: 0.09187458455562592
Epoch 400:
  Total Loss: 0.14453834295272827
  Price Loss: 0.07508152723312378
  First-Order Loss: 0.009067696519196033
  Second-Order Loss: 0.09015632420778275
Epoch 500:
  Total Loss: 0.14682434499263763
  Price Loss: 0.07868623733520508
  First-Order Loss: 0.008473734371364117
  Second-Order Loss: 0.08886642009019852
Epoch 600:
  Total Loss: 0.142159938812

Save all the training records.

In [23]:
dataframes = []
for idx, history in enumerate([history_1,history_2,history_3,history_4], start=1):
    max_length = max(len(v) for v in history.values() if v)

    # Fill empty lists with NaNs
    for key, value in history.items():
        if len(value) < max_length:
            history[key] = value + [np.nan] * (max_length - len(value))
    history['index'] = list(range(len(next(iter(history.values())))))

    # Convert to DataFrame
    df = pd.DataFrame(history)
    df['source'] = f'history{idx}'  # Add a column identifying the source
    dataframes.append(df)

# Combine all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Print the resulting DataFrame
print(combined_df)
combined_df.to_csv("results//learning_history.csv", index=False)


       total_loss  price_loss  first_order_loss  second_order_loss  index   
0        2.947831    2.839680          0.216303                NaN      0  \
1        1.455241    1.346726          0.217028                NaN      1   
2        0.568454    0.459518          0.217870                NaN      2   
3        0.219065    0.109687          0.218756                NaN      3   
4        0.273831    0.164030          0.219602                NaN      4   
...           ...         ...               ...                ...    ...   
21995    0.137554    0.075077          0.009027           0.080227   8995   
21996    0.137543    0.075075          0.009029           0.080210   8996   
21997    0.137531    0.075077          0.009027           0.080194   8997   
21998    0.137520    0.075075          0.009029           0.080178   8998   
21999    0.137508    0.075077          0.009026           0.080161   8999   

         source  lambda1  lambda2  
0      history1      NaN      NaN  
1  

In [31]:
print("Training Time")
print(f"Model 1: {end_time1 - start_time1:f}")
print(f"Model 2: {(end_time2 - start_time2)/3:f}")
print(f"Model 3: {(end_time3 - start_time3)/9:f}")
print(f"Model 4: {(end_time4 - start_time4)/9:f}")

Training Time
Model 1: 142.676085
Model 2: 154.106535
Model 3: 164.432636
Model 4: 169.747968
